##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RNGDet evaluation

## Setup

Install and import the necessary modules.

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import tensorflow as tf
import numpy as np

2023-09-08 08:17:38.281659: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 08:17:38.317879: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:8942] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-08 08:17:38.317907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-08 08:17:38.317926: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-08 08:17:38.324551: I tensorflow/core/platform/cpu_feature_g

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [4]:
from official.projects.rngdet.tasks import rngdet
from official.core import exp_factory
exp_config = exp_factory.get_exp_config('rngdet_cityscale')
task_obj = rngdet.RNGDetTask(exp_config.task)
model = task_obj.build_model()
#task_obj.initialize(model)

/home/ghpark/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-08 08:17:44.337926: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1884] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22275 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:60:00.0, compute capability: 8.6


In [ ]:
print(model.backbone.get_weights()[0][0,0,0])
print("------------------")
print("------------------")
print(model._query_embeddings)

In [5]:
ckpt_dir_or_file = '/home/ghpark/02_RNGDet/ckpt/test_06'
#if tf.io.gfile.isdir(ckpt_dir_or_file):
#  ckpt_dir_or_file = tf.train.latest_checkpoint(ckpt_dir_or_file)
print(ckpt_dir_or_file)
#ckpt = tf.train.Checkpoint(backbone=model.backbone, backbone_history=model.backbone_history)
#ckpt = tf.train.Checkpoint(backbone=model.backbone)
ckpt = tf.train.Checkpoint(
    backbone=model.backbone,
    backbone_history=model.backbone_history,
    transformer=model.transformer,
    query_embeddings=model._query_embeddings,
    segment_head=model._segment_head,
    keypoint_head=model._keypoint_head,
    class_embed=model._class_embed)
status = ckpt.restore(tf.train.latest_checkpoint(ckpt_dir_or_file))
status.expect_partial().assert_existing_objects_matched()
#status.assert_consumed()

/home/ghpark/02_RNGDet/ckpt/test_06


In [ ]:
print(model.backbone.get_weights()[0][0,0,0])
print("------------------")
print("------------------")
print(model._query_embeddings)

In [6]:
from PIL import Image
from official.projects.rngdet.eval import agent

pad_size = 128
sat_image = np.array(Image.open(os.path.join('./region_0_sat.png')))
#sat_image = np.pad(sat_image,np.array(((pad_size,pad_size),(pad_size,pad_size),(0,0))),'constant')
#Image.fromarray(sat_image.astype(np.uint8)).convert('RGB').save(f'./temp.png')

sat_image = tf.cast(sat_image, tf.float32)
agent = agent.Agent(model, sat_image)

2023-09-08 08:17:55.014429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:440] Loaded cuDNN version 8903


1084 initial candidates extracted from the segmentation map...


In [7]:
logit_threshold = 0.75
roi_size = 128

while 1:
    agent.step_counter += 1
    # crop ROI
    sat_ROI, historical_ROI = agent.crop_ROI(agent.current_coord)
    sat_ROI = tf.expand_dims(sat_ROI, 0) / 255.0
    # (gunho) historical_ROI / 255.0 in original code
    historical_ROI = tf.expand_dims(historical_ROI, 0) / 255.0
    historical_ROI = tf.expand_dims(historical_ROI, -1)
    historical_ROI = tf.cast(historical_ROI, tf.float32)
    # predict vertices in the next step
    outputs, pred_segment, pred_keypoint = model(sat_ROI, historical_ROI, training=False)
    # agent moves
    # alignment vertices
    outputs = outputs[-1]
    pred_coords = outputs['box_outputs']
    pred_probs = outputs['cls_outputs']
    alignment_vertices = [[v[0]-agent.current_coord[0]+agent.crop_size//2,
        v[1]-agent.current_coord[1]+agent.crop_size//2] for v in agent.historical_vertices]
    pred_coords_ROI = agent.step(pred_probs,pred_coords,thr=logit_threshold)
    
    if agent.finish_current_image:
        print(f'STEP 3: Finsh exploration. Save visualization and graph...')
        Image.fromarray(
            agent.historical_map[roi_size:-roi_size,roi_size:-roi_size].astype(np.uint8)
            ).convert('RGB').save(f'./segmentation/0_result.png')
        break
    # stop action
        

STEP 3: Finsh exploration. Save visualization and graph...


In [10]:
class Vertex():
    def __init__(self,v,id):
        self.x = v[0]
        self.y = v[1]
        self.id = id
        self.neighbors = []

class Edge():
    def __init__(self,src,dst,id):
        self.src = src
        self.dst = dst
        self.id = id

class Graph():
    def __init__(self):
        self.vertices = {}
        self.edges = {}
        self.vertex_num = 0
        self.edge_num = 0

    def find_v(self,v_coord):
        if f'{v_coord[0]}_{v_coord[1]}' in self.vertices.keys():
            return self.vertices[f'{v_coord[0]}_{v_coord[1]}']
        return 

    def find_e(self,v1,v2):
        if f'{v1.id}_{v2.id}' in self.edges:
            return True
        return None

    def add(self,edge):
        v1_coord = edge[0]
        v2_coord = edge[1]
        v1 = self.find_v(v1_coord)
        if v1 is None:
            v1 = Vertex(v1_coord,self.vertex_num)
            self.vertex_num += 1
            self.vertices[f'{v1.x}_{v1.y}'] = v1
        
        v2 = self.find_v(v2_coord)
        if v2 is None:
            v2 = Vertex(v2_coord,self.vertex_num)
            self.vertex_num += 1
            self.vertices[f'{v2.x}_{v2.y}'] = v2

        if v1 not in v2.neighbors:
            v2.neighbors.append(v1)
        if v2 not in v1.neighbors:
            v1.neighbors.append(v2)
        e = self.find_e(v1,v2)
        if e is None:
            self.edges[f'{v1.id}_{v2.id}'] = Edge(v1,v2,self.edge_num)
            self.edge_num += 1
            self.edges[f'{v2.id}_{v1.id}'] = Edge(v2,v1,self.edge_num)
            self.edge_num += 1



In [11]:
import json
# save generated graph
graph = Graph()
try:
    with open(f'./segmentation/0.json','w') as jf:
        json.dump(agent.historical_edges,jf)
except Exception as e:
    print('Error...')
    print(e)


# TEMP

Use `ds_info` (which is an instance of `tfds.core.DatasetInfo`) to lookup the text descriptions of each class ID.

Run a batch of the processed training data through the model, and view the results